<a href="https://colab.research.google.com/github/MengOonLee/Python/blob/main/Plotly_Dash/Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Beam

In [ ]:
%%writefile app.py
import logging
logging.getLogger().setLevel(logging.INFO)
import csv
import codecs
import datetime
import dash
import plotly.express as px
import apache_beam as beam
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.runners.interactive.interactive_runner \
    import InteractiveRunner

logo_link = 'https://avatars.githubusercontent.com/u/44514389?v=4'
csv_files = ['ecom_sales.csv']

@beam.ptransform_fn
def ReadCsvFiles(pbegin: beam.pvalue.PBegin, file_patterns):

    def expand_pattern(pattern):
        try:
            for m in beam.io.filesystems.FileSystems.match([
                    pattern])[0].metadata_list:
                yield m.path
        except Exception:
            logging.exception(pattern)

    def read_csv_lines(file):
        try:
            with beam.io.filesystems.FileSystems.open(file) as f:
                for row in csv.DictReader(
                        codecs.iterdecode(f, 'utf-8')):
                    yield dict(row)
        except Exception:
            logging.exception(file)

    return pbegin \
        | beam.Create(file_patterns) \
        | beam.FlatMap(expand_pattern) \
        | beam.FlatMap(read_csv_lines)

class ParseSales(beam.DoFn):

    def process(self, elem):
        elem['Quantity'] = int(elem['Quantity'].strip())
        elem['UnitPrice'] = round(
            float(elem['UnitPrice'].strip()), 2)
        elem['InvoiceDate'] = datetime.datetime.strptime(
            elem['InvoiceDate'].strip(), '%m/%d/%Y %H:%M')
        elem['OrderValue'] = round(
            elem['Quantity']*elem['UnitPrice'], 2)
        elem['YearMonth'] = elem['InvoiceDate']\
            .strftime('%Y-%m')
        yield elem

class TopSalesCountryFn(beam.CombineFn):

    def create_accumulator(self):
        country, sales = None, 0.0
        return country, sales

    def add_input(self, accumulator, elem):
        country, sales = accumulator
        if elem['TotalSales'] > sales:
            accumulator = (elem['Country'], elem['TotalSales'])
        return accumulator

    def merge_accumulators(self, accumulators):
        return accumulators[0]

    def extract_output(self, accumulator):
        country, sales = accumulator
        return {'Country':country, 'TotalSales':sales}

with beam.Pipeline(
    runner=InteractiveRunner()
) as pipeline:

    country_list = ['EIRE', 'Netherlands', 'Germany', 'France',
        'Australia']

    sales = pipeline \
        | ReadCsvFiles(csv_files) \
        | beam.Filter(lambda x: x['Country'] in country_list) \
        | beam.ParDo(ParseSales())

    sales_bar = sales \
        | beam.Map(lambda x: beam.Row(**x)) \
        | beam.GroupBy('Country')\
            .aggregate_field('OrderValue', sum, 'TotalSales') \
        | beam.Map(lambda x: x._asdict()) \
        | beam.Map(lambda x: {
            'Country':str(x['Country']),
            'TotalSales': round(float(x['TotalSales']), 2)}) \
        | beam.Reshuffle()

    df_bar = ib.collect(sales_bar)
    fig_bar = px.bar(data_frame=df_bar,
        x='TotalSales', y='Country', color='Country',
        color_discrete_map={'EIRE':'lightblue',
            'Germany':'orange', 'France':'darkblue',
            'Australia':'green', 'Netherlands':'red'})

    top_sales_country = sales_bar \
        | beam.CombineGlobally(TopSalesCountryFn())
    top_country = ib.collect(top_sales_country)['Country'][0]

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    # Add the company logo
    dash.html.Img(src=logo_link),
    dash.html.H1('Sales by Country'),
    dash.html.Div(dash.dcc.Graph(figure=fig_bar),
        style={'width':'750px', 'margin':'auto'}),
    # Add an overall text-containing component
    dash.html.Span(children=[
        # Add the top country text
        'This year, the most sales came from: ',
        dash.html.B(top_country),
        # Italicize copyright notice
        dash.html.I(' Copyright E-Com INC')])
], style={'text-align':'center', 'font-size':22})

if __name__ == '__main__':
    app.run_server(host='0.0.0.0', debug=True)

### Core Dash app

In [ ]:
import dash
import pandas as pd
import plotly.express as px

ecom_sales = pd.read_csv('ecom_sales.csv')
ecom_sales = ecom_sales.groupby(['Year-Month','Country'])['OrderValue']\
    .agg('sum').reset_index(name='Total Sales ($)')

line_fig = px.line(data_frame=ecom_sales, x='Year-Month', y='Total Sales ($)',
    title='Total Sales by Month', color='Country')

# Create the Dash app
app = dash.Dash(__name__)

# Set up the layout with a single graph
app.layout = dash.dcc.Graph(
  id='my-line-graph',
  # Insert the line graph
  figure=line_fig
)

# Set the app to run in development mode
if __name__ == '__main__':
    app.run_server(debug=True)

### HTML

#### Positioning

In [ ]:
import dash

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Div(style={'height':250, 'width':250, 'background-color':'red'}),
    dash.html.Div(children=[
        dash.html.H1('This box'),
        dash.html.H2('Another Title')
    ], style={'height':250, 'width':250, 'background-color':'lightblue'})
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import dash
import plotly.express as px

bar_fig_country = px.bar(data_frame=ecom_sales,
    x='Total Sales ($)', y='Country', color='Country',
    color_discrete_map={'United Kingdom':'lightblue',
        'Germany':'orange', 'France':'darkblue',
        'Australia':'green', 'Hong Kong':'red'}
)

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Div(style={'width':150, 'height':150, 'background-color':'lightblue'}),
    dash.html.H1('Sales Proportion by Country'),
    dash.dcc.Graph(id='bar_graph', figure=bar_fig_country)
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import dash
import pandas as pd
import plotly.express as px

ecom_sales = pd.read_csv('ecom_sales.csv')
ecom_line = ecom_sales.groupby(['Year-Month','Country'])['OrderValue']\
    .agg('sum').reset_index(name='Total Sales ($)')
ecom_bar = ecom_sales.groupby('Country')['OrderValue']\
    .agg('sum').reset_index(name='Total Sales ($)')
line_graph = px.line(data_frame=ecom_line, x='Year-Month', y='Total Sales ($)',
    title='Total Sales by Month', color='Country')
bar_graph = px.bar(data_frame=ecom_bar, x='Total Sales ($)', y='Country',
    orientation='h',title='Total Sales by Country')

# Create the Dash app
app = dash.Dash(__name__)

# Set up the layout using an overall div
app.layout = dash.html.Div(children=[
    # Add a H1
    dash.html.H1('Sales by Country & Over Time'),
    # Add both graphs
    dash.dcc.Graph(id='line_graph', figure=line_graph),
    dash.dcc.Graph(id='bar_graph', figure=bar_graph)
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
ecom_line = ecom_sales.groupby('Year-Month')['OrderValue']\
    .agg('sum').reset_index(name='Total Sales ($)')
line_fig = px.line(data_frame=ecom_line, x='Year-Month', y='Total Sales ($)',
    title='Total Sales by Month')
ecom_bar = ecom_sales.groupby('Country')['OrderValue']\
    .agg('sum').reset_index(name='Total Sales ($)')
max_country = ecom_bar.sort_values(by='Total Sales ($)', ascending=False)\
    .loc[0]['Country']
bar_fig = px.bar(data_frame=ecom_bar, x='Total Sales ($)', y='Country',
    orientation='h', title='Total Sales by Country')

app = dash.Dash(__name__)

# Create the dash layout and overall div
app.layout = dash.html.Div(children=[
    dash.html.H1('Sales Figures'),
    # Add a div containing the line figure
    dash.html.Div(dcc.Graph(id='my-line-fig', figure=line_fig)),
    # Add a div containing the bar figure
    dash.html.Div(dcc.Graph(id='my-bar-fig',figure=bar_fig)),
    # Add the H3
    html.H3(f'The largest country by sales was {max_country}')
])

if __name__ == '__main__':
    app.run_server(debug=True)

#### Font styles

In [ ]:
import dash

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Img(src='www.website.com/logo.png'),
    dash.html.H1('Our Sales Dashboard'),
    dash.html.Span(children=[
        f'Prepared:{datetime.now().date()}',
        dash.html.Br(),
        ' by ', dash.html.B('Jessie Parker, '),
        dash.html.Br(),
        dash.html.I('Data Scientist')
    ])
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e_com_logo.png'

ecom_bar = ecom_sales.groupby('Country')['OrderValue']\
    .agg('sum').reset_index(name='Total Sales ($)')\
    .sort_values(by='Total Sales ($)', ascending=False)
top_country = ecom_bar.loc[0]['Country']
bar_fig_country = px.bar(ecom_bar, x='Total Sales ($)', y='Country', color='Country',
        color_discrete_map={'United Kingdom':'lightblue', 'Germany':'orange',
        'France':'darkblue', 'Australia':'green', 'Hong Kong':'red'}
)

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    # Add the company logo
    dash.html.Img(src=logo_link),
    dash.html.H1('Sales by Country'),
    dash.html.Div(dash.dcc.Graph(figure=bar_fig_country),
        style={'width':'750px', 'margin':'auto'}),
    # Add an overall text-containing component
    dash.html.Span(children=[
        # Add the top country text
        'This year, the most sales came from: ',
        dash.html.B(top_country),
        # Italicize copyright notice
         dash.html.I('Copyright E-Com INC')
    ])
], style={'text-align':'center', 'font-size':22})

if __name__ == '__main__':
    app.run_server(debug=True)

#### Bullet List

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo_white.png'
ecom_category = ecom_sales.groupby(['Major Category','Minor Category'])\
    .size().reset_index(name='Total Orders')\
    .sort_values(by='Total Orders', ascending=False)\
    .reset_index(drop=True)
num1_cat, num1_salesvol = ecom_category.loc[0].tolist()[1:3]
num2_cat, num2_salesvol = ecom_category.loc[1].tolist()[1:3]
ecom_bar = px.bar(data_frame=ecom_category,
    x='Total Orders', y='Minor Category', color='Major Category')
ecom_bar.update_layout({
    'yaxis': {'dtick':1, 'categoryorder':'total ascending'}
})

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Img(src=logo_link),
    dash.html.H1('Top Sales Categories'),
    dash.html.Div(dash.dcc.Graph(figure=ecom_bar)),
    dash.html.Span(children=[
        'The top 2 sales categories were:',
        # Set up an ordered list
        dash.html.Ol(children=[
            # Add two list elements with the top category variables
            dash.html.Li(children=[num1_cat, ' with ',
                num1_salesvol, ' sales volume']),
            dash.html.Li(children=[num2_cat, ' with ',
                num2_salesvol, ' sales volume'])
    ], style={'width':'350px', 'margin':'auto'}),
    # Add a line break before the copyright notice
    dash.html.Br(),
    dash.html.I('Copyright E-Com INC')])
], style={'text-align':'center', 'font-size':22})

if __name__ == '__main__':
    app.run_server(debug=True)

### CSS

#### Style

In [ ]:
import dash

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.H1('Welcome to the website!'),
    dash.html.H2('Text', style={'font-size':'50px', 'color':'red'}),
    # Add & resize the company logo
    dash.html.Img(src=logo_link, style={'width':'250px', 'height':'250px'}),
    dash.html.Div(dash.dcc.Graph(figure=ecom_bar),
        style={'width':'500px', 'height':'450px', 'border':'5px dotted red'})
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e_com_logo.png'

ecom_category = ecom_sales.groupby(['Major Category','Minor Category'])\
    .size().reset_index(name='Total Orders')\
    .sort_values(by='Total Orders', ascending=False)\
    .reset_index(drop=True)
top_cat = ecom_category.loc[0]['Minor Category']
ecom_bar = px.bar(ecom_category, x='Total Orders', y='Minor Category',
    color='Major Category')
ecom_bar.update_layout({
    'yaxis':{'dtick':1, 'categoryorder':'total ascending'},
    'paper_bgcolor':'rgb(224, 255, 252)'
})

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    # Set the new white-text image
    dash.html.Img(src=logo_link,
        # Set the size of the logo
        style={'width':'165px', 'height':'50px'}),
    dash.html.H1('Top Sales Categories'),
    # Set the size of the bar graph
    dash.html.Div(dash.dcc.Graph(figure=ecom_bar,
        style={'width':'500px', 'height':'350px', 'margin':'auto'})),
    dash.html.Br(),
    dash.html.Span(children=[
        'The top category was: ',
        dash.html.B(top_cat),
        dash.html.Br(),
        dash.html.I('Copyright E-Com INC',
            # Add a background color to the copyright notice
            style={'background-color':'lightgrey'})])
], style={'text-align':'center', 'font-size':22,
    # Update the background color to the entire app
    'background-color':'black',
    # Change the text color for the whole app
    'color':'white'})

app.layout = dash.html.Div(children=[
    dash.html.Img(src=logo_link,
        # Set the size of the logo
        style={'width':'215px', 'height':'240px'}
    ),
    dash.html.H1('Top Sales Categories'),
    # Set the size of the bar graph
    dash.html.Div(dash.dcc.Graph(figure=ecom_bar,
        style={'width':'500px', 'height':'350px', 'margin':'auto'})),
    dash.html.Br(),
    dash.html.Span(children=[
        'The top category was: ',
        dash.html.B(top_cat),
        dash.html.Br(),
        dash.html.I('Copyright E-Com INC',
            # Add a background color to the copyright notice
            style={'background-color':'lightgrey'})
    ])
# Add a background color to the entire app
], style={'text-align':'center', 'font-size':22, 'background-color':'rgb(224, 255, 252)'})

if __name__ == '__main__':
    app.run_server(debug=True)

#### Darkmode

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo_white.png'
ecom_category = ecom_sales.groupby(
        ['Major Category','Minor Category'])\
    .size().reset_index(name='Total Orders')\
    .sort_values(by='Total Orders', ascending=False)\
    .reset_index(drop=True)
top_cat = ecom_category.loc[0]['Minor Category']
ecom_bar = px.bar(data_frame=ecom_category,
    x='Total Orders', y='Minor Category', color='Major Category')

# Set the font color of the bar chart
ecom_bar.update_layout({
    'yaxis':{'dtick':1, 'categoryorder':'total ascending'},
    'paper_bgcolor':'black', 'font': {'color':'white'}
})

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    # Set the new white-text image
    dash.html.Img(src=logo_link,
        # Set the size of the logo
        style={'width':'165px', 'height':'50px'}),
    dash.html.H1('Top Sales Categories'),
    # Set the size of the bar graph
    dash.html.Div(dash.dcc.Graph(figure=ecom_bar,
        style={'width':'500px', 'height':'350px', 'margin':'auto'})),
    dash.html.Br(),
    dash.html.Span(children=[
        'The top category was: ',
        dash.html.B(top_cat),
        dash.html.Br(),
        dash.html.I('Copyright E-Com INC',
            # Add a background color to the copyright notice
            style={'background-color':'lightgrey'})])
], style={'text-align':'center', 'font-size':22,
    # Update the background color to the entire app
    'background-color':'black',
    # Change the text color for the whole app
    'color':'white'})

if __name__ == '__main__':
    app.run_server(debug=True)

#### Inline Placement

In [ ]:
import dash
import plotly.express as px
import pandas as pd

logo_link = 'https://assets.datacamp.com/e-comlogo.png'
ecom_sales = pd.read_csv('ecom_sales.csv')
ecom_line = ecom_sales.groupby('Year-Month')['OrderValue']\
    .agg('sum').reset_index(name='TotalSales')
line_fig = px.line(data_frame=ecom_line,
    x='Year-Month', y='TotalSales', title='Total Sales by Month')
line_fig.update_layout({'paper_bgcolor':'rgb(224, 255, 252)'})
ecom_bar = ecom_sales.groupby('Country')['OrderValue']\
    .agg('sum').reset_index(name='TotalSales')
bar_fig = px.bar(data_frame=ecom_bar, x='TotalSales', y='Country',
    orientation='h', title='Total Sales by Country')
bar_fig.update_layout({
    'yaxis':{'dtick':1, 'categoryorder':'total ascending'},
    'paper_bgcolor':'rgb(224, 255, 252)'
})

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Div(children=[
        html.Img(src=logo_link,
            # Place the logo side-by-side the H1 with required margin
            style={'display':'inline-block', 'margin':'25px'}
        ),
        dash.html.H1(children=['Sales Figures'],
            # Make the H1 side-by-side with the logos
            style={'display':'inline-block'}
        ),
        dash.html.Img(src=logo_link,
            # Place the logo side-by-side the H1 with required margin
            style={'display':'inline-block', 'margin':'25px'})
    ]),
    dash.html.Div(dash.dcc.Graph(figure=line_fig),
        # Ensure graphs are correct size, side-by-side with required margin
        style={'width':'500px', 'display':'inline-block', 'margin':'5px'}
    ),
    dash.html.Div(dash.dcc.Graph(figure=bar_fig),
        # Ensure graphs are correct size, side-by-side with required margin
        style={'width':'350px', 'display':'inline-block', 'margin':'5px'}
    ),
    dash.html.H3(f'The largest order quantity was {ecom_sales.Quantity.max()}')
], style={'text-align':'center', 'font-size':22, 'background-color':'rgb(224, 255, 252)'})

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo.png'
ecom_bar_major_cat = ecom_sales.groupby('Major Category')['OrderValue']\
    .agg('sum').reset_index(name='Total Sales ($)')
ecom_bar_minor_cat = ecom_sales.groupby('Minor Category')['OrderValue']\
    .agg('sum').reset_index(name='Total Sales ($)')
bar_fig_major_cat = px.bar(data_frame=ecom_bar_major_cat,
    x='Total Sales ($)', y='Major Category', color='Major Category',
    color_discrete_map={'Clothes':'blue', 'Kitchen':'red', 'Garden':'green',
        'Household':'yellow'}
)
bar_fig_minor_cat = px.bar(data_frame=ecom_bar_minor_cat,
    x='Total Sales ($)', y='Minor Category')

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Img(src=logo_link,
        # Add margin to the logo
        style={'margin':'30px 0px 0px 0px'}),
    dash.html.H1('Sales breakdowns'),
    dash.html.Div(children=[
        dash.dcc.Graph(
            # Style the graphs to appear side-by-side
            figure=bar_fig_major_cat, style={'display':'inline-block'}),
        dash.dcc.Graph(figure=bar_fig_minor_cat, style={'display':'inline-block'})
    ]),
    dash.html.H2('Major Category',
        # Style the titles to appear side-by-side with a 2 pixel border
        style={'display':'inline-block', 'border':'2px solid black',
        # Style the titles to have the correct spacings
            'padding':'10px', 'margin':'10px 220px'}
    ),
    dash.html.H2('Minor Category',
        # Style the titles to appear side-by-side with a 2 pixel border
        style={'display':'inline-block', 'border':'2px solid black',
        # Style the titles to have the correct spacings
            'padding':'10px', 'margin':'10px 220px'}
    )
], style={'text-align':'center', 'font-size':22})

if __name__ == '__main__':
    app.run_server(debug=True)

### Callbacks

#### Dropdown

In [ ]:
import dash
import plotly.express as px

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.dcc.Dropdown(id='title_dd',
        options=[
            {'label':'title1', 'value':'title1'},
            {'label':'title2', 'value':'title2'}
        ]
    ),
    dash.dcc.Graph(id='my_graph')
])

@dash.callback(
    dash.Output(component_id='my_graph', component_property='figure'),
    dash.Input(component_id='title_dd', component_property='value')
)
def update_plot(selection):
    title = "None Selected"
    if selection:
        title = selection
    bar_fig = px.bar(data_frame=ecom_sales,
        x='Total Sales ($)', y='Country',
        title=f'{title}'
    )
    return bar_fig

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import pandas as pd
import dash
import plotly.express as px

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo.png'

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Img(src=logo_link, style={'margin':'30px 0px 0px 0px'}),
    dash.html.H1('Sales breakdowns'),
    dash.html.Div(children=[
        dash.html.Div(children=[
            dash.html.H2('Controls'),
            dash.html.Br(),
            dash.html.H3('Country Select'),
            # Add a dropdown with identifier
            dash.dcc.Dropdown(id='country_dd',
            # Set the available options with noted labels and values
                options=[
                    {'label':'UK', 'value':'United Kingdom'},
                    {'label':'GM', 'value':'Germany'},
                    {'label':'FR', 'value':'France'},
                    {'label':'AUS', 'value':'Australia'},
                    {'label':'HK', 'value':'Hong Kong'}
                ], style={'width':'200px', 'margin':'0 auto'})
        ], style={'width':'350px', 'height':'350px',
            'display':'inline-block', 'vertical-align':'top',
            'border':'1px solid black', 'padding':'20px'}),
        dash.html.Div(children=[
            # Add a graph component with identifier
            dash.dcc.Graph(id='major_cat'),
            dash.html.H2('Major Category',
                style={'border':'2px solid black', 'width':'200px', 'margin':'0 auto'})
        ], style={'width':'700px', 'display':'inline-block'})
    ])
], style={'text-align':'center', 'display':'inline-block', 'width':'100%'})

@dash.callback(
    # Set the input and output of the callback to link the dropdown to the graph
    dash.Output(component_id='major_cat', component_property='figure'),
    dash.Input(component_id='country_dd', component_property='value')
)

def update_plot(input_country):
    # Set a default value
    country_filter = 'All Countries'
    # Ensure the DataFrame is not overwritten
    sales = ecom_sales.copy(deep=True)
    # Create a conditional to filter the DataFrame if the input exists
    if input_country:
        country_filter = input_country
        sales = sales[sales['Country']==country_filter]
    ecom_bar_major_cat = sales.groupby('Major Category')['OrderValue']\
        .agg('sum').reset_index(name='Total Sales ($)')
    bar_fig_major_cat = px.bar(data_frame=ecom_bar_major_cat,
        x='Total Sales ($)', y='Major Category',
        color='Major Category', title=f'Sales in {country_filter}',
        color_discrete_map={'Clothes':'blue', 'Kitchen':'red',
            'Garden':'green','Household':'yellow'})
    # Return the figure
    return bar_fig_major_cat

if __name__ == '__main__':
    app.run_server(debug=True)

#### Date picker

In [ ]:
import dash
import plotly.express as px
import pandas as pd
import datetime

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo.png'
ecom_sales['InvoiceDate'] = pd.to_datetime(ecom_sales['InvoiceDate'])

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Img(src=logo_link, style={'margin':'30px 0px 0px 0px'}),
    dash.html.H1('Sales breakdowns'),
    dash.html.Div(children=[
        dash.html.Div(children=[
            dash.html.H2('Controls'),
            dash.html.Br(),
            dash.html.H3('Sale Date Select'),
            # Create a single date picker with identifier
            dash.dcc.DatePickerSingle(id='sale_date',
                # Set the min/max dates allowed as the min/max dates in the DataFrame
                min_date_allowed=ecom_sales['InvoiceDate'].min(),
                max_date_allowed=ecom_sales['InvoiceDate'].max(),
                # Set the initial visible date
                date=datetime.date(2011, 4, 11),
                initial_visible_month=datetime.date(2011, 4, 11),
                style={'width':'200px', 'margin':'0 auto'})
        ], style={'width':'350px', 'height':'350px',
            'display':'inline-block', 'vertical-align':'top',
            'border':'1px solid black', 'padding':'20px'}),
        dash.html.Div(children=[
            # Add a component to render a Plotly figure with the specified id
            dash.dcc.Graph(id='sales_cat'),
            dash.html.H2('Daily Sales by Major Category',
                style={'border':'2px solid black', 'width':'400px', 'margin':'0 auto'})
        ], style={'width':'700px','display':'inline-block'})
    ])
], style={'text-align':'center', 'display':'inline-block', 'width':'100%'})
# Create a callback and link
@dash.callback(
    dash.Output(component_id='sales_cat', component_property='figure'),
    dash.Input(component_id='sale_date', component_property='date')
)
def update_plot(input_date):
    # Ensure the DataFrame is not overwritten
    sales = ecom_sales.copy(deep=True)
    # Conditionally filter the DataFrame using the input
    if input_date:
        sales = sales[sales['InvoiceDate']==input_date]
    ecom_bar_major_cat = sales.groupby('Major Category')['OrderValue']\
        .agg('sum').reset_index(name='Total Sales ($)')
    bar_fig_major_cat = px.bar(data_frame=ecom_bar_major_cat,
        x='Total Sales ($)', y='Major Category',
        title=f'Sales on: {input_date}', orientation='h')
    # Return the figure to render
    return bar_fig_major_cat

if __name__ == '__main__':
    app.run_server(debug=True)

#### Slider

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo.png'

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Img(src=logo_link, style={'margin':'30px 0px 0px 0px'}),
    dash.html.H1('Sales breakdowns'),
    dash.html.Div(children=[
        dash.html.Div(children=[
            dash.html.H2('Controls'),
            dash.html.Br(),
            dash.html.H3('Minimum OrderValue Select'),
            # Add a slider input
            dash.dcc.Slider(id='value_slider',
                # Set the min and max of the slider
                min=ecom_sales['OrderValue'].min(),
                max=ecom_sales['OrderValue'].max(),
                # Start the slider at 0
                value=0,
                # Increment the slider by 50 each notch
                step=50,
                vertical=False)
        ], style={'width':'350px', 'height':'350px',
            'display':'inline-block', 'vertical-align':'top',
            'border':'1px solid black', 'padding':'20px'}),
        dash.html.Div(children=[
            dash.dcc.Graph(id='sales_cat'),
            dash.html.H2('Sales by Major Category',
                style={'border':'2px solid black', 'width':'400px', 'margin':'0 auto'})
        ], style={'width':'700px', 'display':'inline-block'}),
    ])
], style={'text-align':'center', 'display':'inline-block', 'width':'100%'})

@dash.callback(
    dash.Output(component_id='sales_cat', component_property='figure'),
    dash.Input(component_id='value_slider', component_property='value')
)
def update_plot(min_val):
    sales = ecom_sales.copy(deep=True)
    if min_val:
        sales = sales[sales['OrderValue']>=min_val]
    ecom_bar_major_cat = sales.groupby('Major Category')['OrderValue']\
        .size().reset_index(name='Total Sales Volume')
    bar_fig_major_cat = px.bar(data_frame=ecom_bar_major_cat,
        x='Total Sales Volume', y='Major Category',
        title=f'Sales with order value: {min_val}', orientation='h')

    return bar_fig_major_cat

if __name__ == '__main__':
    app.run_server(debug=True)

#### DRY-styling Dash components

In [ ]:
import dash
import plotly.express as px
import pandas as pd
import datetime

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo.png'
ecom_sales['InvoiceDate'] = pd.to_datetime(ecom_sales['InvoiceDate'])

app = dash.Dash(__name__)

def make_break(num_breaks):
    br_list = [dash.html.Br()] * num_breaks
    return br_list

# Create a reusable component function called add_logo
def add_logo():
    # Add a component that will render an image
    corp_logo = dash.html.Img(src=logo_link,
        # Add the corporate styling
        style={'margin':'20px 20px 5px 5px',
            'padding':'50px 50px',
            'border':'1px dashed lightblue',
            'background-color':'rgb(230, 131, 247)',
            'display':'inline-block'})
    return corp_logo

# Create a function to add corporate styling
def style_c():
    layout_style={'display':'inline-block', 'margin':'0 auto', 'padding':'20px',
        'border':'2px solid black'}
    return layout_style

app.layout = dash.html.Div(children=[
    # Insert the logo (1)
    add_logo(),
    # Insert two HTML breaks
    *make_break(2),
    dash.html.H1('Sales breakdowns'),
    # Insert three HTML breaks
    *make_break(3),
    dash.html.Div(children=[
        dash.html.Div(children=[
            # Insert the logo (2)
            add_logo(),
            # Style using the styling function
            dash.html.H2('Controls', style=style_c()),
            # Insert the logo (3)
            add_logo(),
            dash.html.H3('Sale Date Select'),
            # Insert two HTML breaks
            *make_break(2),
            dash.dcc.DatePickerSingle(id='sale_date',
                min_date_allowed=ecom_sales.InvoiceDate.min(),
                max_date_allowed=ecom_sales.InvoiceDate.max(),
                initial_visible_month=datetime.date(2011, 4, 1),
                date=datetime.date(2011, 4, 11),
                # Add styling using the styling function
                style={'width':'200px'}.update(style_c())),
        ], style={'width':'550px', 'height':'350px', 'vertical-align':'top',
            'border':'1px solid black'}.update(style_c())),
        dash.html.Div(children=[
            dash.dcc.Graph(id='sales_cat'),
            dash.html.H2('Daily Sales by Major Category',
                style={'border':'2px solid black', 'width':'400px', 'margin':'0 auto'})
        ], style={'width':'700px','display':'inline-block'})
    ]),
    # Insert the logo (4)
    add_logo()
], style={'text-align':'center', 'display':'inline-block', 'width':'100%'})

@dash.callback(
    dash.Output(component_id='sales_cat', component_property='figure'),
    dash.Input(component_id='sale_date', component_property='date')
)
def update_plot(input_date):

    sales = ecom_sales.copy(deep=True)
    if input_date:
        sales = sales[sales['InvoiceDate']==input_date]
    ecom_bar_major_cat = sales.groupby('Major Category')['OrderValue']\
        .agg('sum').reset_index(name='Total Sales ($)')
    bar_fig_major_cat = px.bar(data_frame=ecom_bar_major_cat,
        x='Total Sales ($)', y='Major Category',
        title=f'Sales on: {input_date}', orientation='h')

    return bar_fig_major_cat

if __name__ == '__main__':
    app.run_server(debug=True)

#### User inputs

In [ ]:
import dash
import plotly.express as px
import pandas as pd

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.dcc.Input(id='my_input', type='text', placeholder='Enter your text',
        debounce=False),
    dash.dcc.Graph(id='my_graph')
])

@dash.callback(
    dash.Output(component_id='my_graph', component_property='figure'),
    dash.Input(component_id='my_input', component_property='value')
)
def update_plot(my_input):
    df = data.copy(deep=True)
    df = df[df['column']==my_input]
    fig = px.scatter(data_frame=df)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo.png'

def make_break(num_breaks):
    br_list = [dash.html.Br()] * num_breaks
    return br_list

def add_logo():
    corp_logo = dash.html.Img(src=logo_link, style={
        'margin':'20px 20px 5px 5px', 'border':'1px dashed lightblue',
        'display':'inline-block'})
    return corp_logo

def style_c():
    layout_style={'display':'inline-block', 'margin':'0 auto', 'padding':'20px'}
    return layout_style

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    add_logo(),
    *make_break(2),
    dash.html.H1('Sales Dashboard'),
    *make_break(3),
    dash.html.Div(children=[
        dash.html.Div(children=[
            dash.html.H2('Controls', style=style_c()),
            dash.html.H3('Search Descriptions'),
            *make_break(2),
            # Add the required input
            dash.dcc.Input(id='search_desc', type='text',
                placeholder='Filter Product Descriptions',
                # Ensure input is triggered with 'Enter'
                debounce=True,
                # Ensure the plot can load without a selection
                required=False,
                style={'width':'200px', 'height':'30px'})
        ], style={'width':'350px', 'height':'350px',
            'vertical-align':'top', 'border':'1px solid black',
            'display':'inline-block', 'margin':'0px 80px'}),
        dash.html.Div(children=[
            dash.dcc.Graph(id='sales_desc'),
            dash.html.H2('Sales Quantity by Country',
                style={'border':'2px solid black', 'width':'400px', 'margin':'0 auto'})
        ], style={'width':'700px', 'display':'inline-block'})
    ])
], style={'text-align':'center', 'display':'inline-block', 'width':'100%'})

@dash.callback(
    dash.Output(component_id='sales_desc', component_property='figure'),
    dash.Input(component_id='search_desc', component_property='value')
)
def update_plot(search_value):
    title_value = 'None Selected (Showing all)'

    sales = ecom_sales.copy(deep=True)

    # Undertake the filter here using the user input
    if search_value:
        sales = sales[sales['Description'].str.contains(search_value, case=False)]
        title_value = search_value

    fig = px.scatter(data_frame=sales,
        x='Quantity', y='OrderValue', color='Country',
        title=f'Sales with description text: {title_value}')

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo.png'

def make_break(num_breaks):
    br_list = [dash.html.Br()] * num_breaks
    return br_list

def add_logo():
    corp_logo = dash.html.Img(src=logo_link, style={
        'margin':'20px 20px 5px 5px', 'border':'1px dashed lightblue',
        'display':'inline-block'})
    return corp_logo

def style_c():
    layout_style={'display':'inline-block', 'margin':'0 auto', 'padding':'20px'}
    return layout_style

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    add_logo(),
    *make_break(2),
    dash.html.H1('Sales Dashboard'),
    *make_break(3),
    dash.html.Div(children=[
        dash.html.Div(children=[
            dash.html.H2('Controls', style=style_c()),
            dash.html.H3('Set minimum OrderValue'),
            *make_break(2),
            dash.dcc.Input(
                # Create the specified input
                id='min_order_val', type='range',
                min=50, max=550, value=50,
                # Ensure the callback is triggered only
                # when the user stops moving the slider
                debounce=False,
                style={'width':'300px', 'height':'30px'})
        ], style={'width':'350px', 'height':'350px',
            'vertical-align':'top', 'border':'1px solid black',
            'display':'inline-block', 'margin':'0px 80px'}),
        dash.html.Div(children=[
            dash.dcc.Graph(id='sales_country'),
            dash.html.H2('Sales Quantity by Country',
                style={'border':'2px solid black', 'width':'400px', 'margin':'0 auto'})
        ], style={'width':'700px','display':'inline-block'})
    ])
], style={'text-align':'center', 'display':'inline-block', 'width':'100%'})

@dash.callback(
    dash.Output(component_id='sales_country', component_property='figure'),
    dash.Input(component_id='min_order_val', component_property='value')
)
def update_plot(input_val):

    if not input_val:
        input_val = 0

    sales = ecom_sales.copy(deep=True)

    # Check for input then use to subset data
    if input_val:
        input_val = round(float(input_val), 2)
        sales = sales[sales['OrderValue'] > input_val]

    fig = px.scatter(data_frame=sales,
        x='Quantity', y='OrderValue', color='Country', height=400,
        # Set the conditional title
        title=f'Orders of Min Value ${input_val}')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

#### Data table

In [ ]:
import dash

d_columns = [
    {'name':'Major Category', 'id':'Major Category'},
    {'name':'Total Sales ($)', 'id':'Total Sales ($)', 'type':'numeric',
        'format':dash.dash_table.FormatTemplate.money(2)},
    {'name':'Sales Volume', 'id':'Sales Volume'}
]

d_table = dash.dash_table.DataTable(
    columns=d_columns,
    data=dataframe.to_dict('records'),
    cell_selectable=False,
    # Add sort ability
    sort_action='native',
    # Add filter ability
    filter_action='native',
    # Add pagination
    page_current=0, page_size=2, page_action='native'
)

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo.png'

key_stats_tb = ecom_sales.groupby(['Country', 'Major Category', 'Minor Category'])\
    ['OrderValue'].agg(['sum', 'count', 'mean']).reset_index()\
    .rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)',
        'mean':'Average Order Value ($)'})

# Set up money format for relevant columns
money_format = dash.dash_table.FormatTemplate.money(2)
money_cols = ['Total Sales ($)', 'Average Order Value ($)']
data_cols = [x for x in key_stats_tb.columns if x not in money_cols]
d_columns = [{'name':x, 'id':x} for x in data_cols]
d_columns += [{'name':x, 'id':x, 'type':'numeric', 'format':money_format}
    for x in money_cols]

# Create a Data Table
d_table = dash.dash_table.DataTable(columns=d_columns,
    data=key_stats_tb.to_dict('records'), cell_selectable=False,
    # Add sorting
    sort_action='native',
    # Add filtering
    filter_action='native',
    # Add pagination
    page_action='native',
    # Start on the first page
    page_current=0,
    # Render 7 items per page
    page_size=7
)

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Img(src=logo_link, style={'margin':'30px 0px 0px 0px'}),
    dash.html.H1('Sales Aggregations'),
    dash.html.Div(children=[
        dash.html.H2('Key Aggregations'),
        d_table
    ], style={'width':'850px', 'height':'750px', 'margin':'0 auto'}),
], style={'text-align':'center', 'display':'inline-block', 'width':'100%'})

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo.png'

major_categories = list(ecom_sales['Major Category'].unique())
large_tb = ecom_sales.groupby(['Major Category', 'Minor Category'])['OrderValue']\
    .agg(['sum', 'count', 'mean']).reset_index()\
    .rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)',
        'mean':'Average Order Value ($)'})
ecom_country = ecom_sales.groupby('Country')['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')
bar_fig_country = px.bar(data_frame=ecom_country,
    x='Total Sales ($)', y='Country', width=500, height=450,
    title='Total Sales by Country (Hover to filter the Minor Category bar chart!)',
    custom_data=['Country'], color='Country',
    color_discrete_map={'United Kingdom':'lightblue', 'Germany':'orange',
        'France':'darkblue', 'Australia':'green', 'Hong Kong':'red'})

money_format = dash.dash_table.FormatTemplate.money(2)
money_cols = ['Total Sales ($)', 'Average Order Value ($)']
d_columns = [{'name':x, 'id':x} for x in large_tb.columns if x not in money_cols]
d_columns += [
    {'name':'Total Sales ($)', 'id':'Total Sales ($)', 'type':'numeric',
        'format':money_format},
    {'name':'Average Order Value ($)', 'id':'Average Order Value ($)', 'type':'numeric',
        'format':money_format}
]

d_table = dash.dash_table.DataTable(
    # Set up the columns and data
    columns=d_columns, data=large_tb.to_dict('records'), cell_selectable=False,
    # Set up sort, filter and pagination
    sort_action='native', filter_action='native', page_action='native', page_current= 0,
    page_size= 7
)

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Img(src=logo_link, style={'margin':'30px 0px 0px 0px'}),
    dash.html.H1('Sales breakdowns'),
    dash.html.Div(children=[
        dash.html.Div(children=[
            dash.html.H2('Controls'),
            dash.html.Br(),
            dash.html.H3('Major Category Select'),
            dash.dcc.Dropdown(id='major_cat_dd',
                options=[{'label':category, 'value':category}
                    for category in major_categories],
                style={'width':'200px', 'margin':'0 auto'}),
            dash.html.Br(),
            dash.html.H3('Minor Category Select'),
            dash.dcc.Dropdown(id='minor_cat_dd',
                style={'width':'200px', 'margin':'0 auto'})
        ], style={'width':'350px', 'height':'360px', 'display':'inline-block',
            'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
        dash.html.Div(children=[
            dash.html.H3(id='chosen_major_cat_title'),
            dash.dcc.Graph(id='sales_line')
        ], style={'width':'700px', 'height':'380px','display':'inline-block',
            'margin-bottom':'5px'})
    ]),
    dash.html.Div(
        # Insert the Data Table
        d_table, style={'width':'1000px', 'height':'350px', 'margin':'10px auto',
            'padding-right':'30px'}),
    dash.html.Div(children=[
        dash.html.Div(dash.dcc.Graph(id='major_cat', figure=bar_fig_country),
            style={'display':'inline-block'}),
        dash.html.Div(dcc.Graph(id='minor_cat'), style={'display':'inline-block'})
    ], style={'width':'1000px', 'height':'650px','display':'inline-block'})
], style={'text-align':'center', 'display':'inline-block', 'width':'100%'})

# Set up a callback with multiple outputs
@dash.callback(
   dash.Output(component_id='minor_cat_dd', component_property='options'),
   dash.Output(component_id='chosen_major_cat_title', component_property='children'),
   dash.Input(component_id='major_cat_dd', component_property='value')
)
def update_dd(major_cat_dd):
    major_minor = ecom_sales[['Major Category', 'Minor Category']].drop_duplicates()
    relevant_minor = major_minor[major_minor['Major Category']==major_cat_dd]\
        ['Minor Category'].values.tolist()
    minor_options = [dict(label=x, value=x) for x in relevant_minor]

    if not major_cat_dd:
        major_cat_dd = 'ALL'

    major_cat_title = f'This is in the Major Category of : {major_cat_dd}'

    return minor_options, major_cat_title

@dash.callback(
    dash.Output(component_id='sales_line', component_property='figure'),
    dash.Input(component_id='minor_cat_dd', component_property='value')
)
def update_line(minor_cat):
    minor_cat_title = 'All'
    ecom_line = ecom_sales.copy()
    if minor_cat:
        minor_cat_title = minor_cat
        ecom_line = ecom_line[ecom_line['Minor Category']==minor_cat]
    ecom_line = ecom_line.groupby('Year-Month')['OrderValue'].agg('sum')\
        .reset_index(name='Total Sales ($)')
    line_graph = px.line(data_frame=ecom_line, x='Year-Month', y='Total Sales ($)',
        title=f'Total Sales by Month for Minor Category: {minor_cat_title}', height=350)

    return line_graph

@dash.callback(
    dash.Output(component_id='minor_cat', component_property='figure'),
    dash.Input(component_id='major_cat', component_property='hoverData')
)
def update_min_cat_hover(hoverData):
    hover_country = 'Australia'

    if hoverData:
        hover_country = hoverData['points'][0]['customdata'][0]

    minor_cat_df = ecom_sales[ecom_sales['Country']==hover_country]
    minor_cat_agg = minor_cat_df.groupby('Minor Category')['OrderValue']\
        .agg('sum').reset_index(name='Total Sales ($)')
    ecom_bar_minor_cat = px.bar(data_frame=minor_cat_agg,
        x='Total Sales ($)', y='Minor Category', orientation='h', height=450, width=480,
        title=f'Sales by Minor Category for: {hover_country}')
    ecom_bar_minor_cat.update_layout({
        'yaxis':{'dtick':1, 'categoryorder':'total ascending'},
        'title':{'x':0.5}
    })

    return ecom_bar_minor_cat

if __name__ == '__main__':
    app.run_server(debug=True)

### Interactivity

#### Between-elements

In [ ]:
import dash
import plotly.express as px

ecom_scatter = px.scatter(data_frame=ecom_data,
    x='Total Sales ($)', y='Sales Volume', color='Country',
    custom_data=['Country'])

app = dash.Dash(__name__)

# Create figure & dash.Dash()
app.layout = dash.html.Div(children=[
    dash.dcc.Graph(id='bar_fig', figure=ecom_scatter),
    dash.html.Br(),
    dash.html.H2('The Hover Data:'),
    dash.html.P(id='text_output')
])

@dash.callback(
    dash.Output(component_id='text_output', component_property='children'),
    dash.Input(component_id='bar_fig', component_property='clickData')
)
def capture_hover_data(clickData):
    return str(clickData)

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo.png'

ecom_country = ecom_sales.groupby('Country')['OrderValue']\
    .agg(['sum', 'count']).reset_index()\
    .rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)'})

# Add the country data to the scatter plot
ecom_scatter = px.scatter(data_frame=ecom_country,
    x='Total Sales ($)', y='Sales Volume', color='Country',
    width=350, height=400, custom_data=['Country'])
ecom_scatter.update_layout({
    'legend':dict(orientation='h', y=-0.5, x=1, yanchor='bottom', xanchor='right'),
    'margin':dict(l=20, r=20, t=25, b=0)})

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Img(src=logo_link, style={'margin':'30px 0px 0px 0px'}),
    dash.html.H1('Sales breakdowns'),
    dash.html.Div(children=[
        dash.html.Div(children=[
            dash.html.H2('Sales by Country'),
            dash.dcc.Graph(id='scatter_fig', figure=ecom_scatter)
        ], style={'width':'350px', 'height':'500px', 'display':'inline-block',
            'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
        dash.html.Div(children=[
            dash.html.H2('Key Stats'),
            dash.html.P(id='text_output', style={'width':'500px', 'text-align':'center'}),
        ], style={'width':'700px', 'height':'650px', 'display':'inline-block'})
    ])
], style={'text-align':'center', 'display':'inline-block', 'width':'100%'})

# Trigger callback on hover
@dash.callback(
    dash.Output(component_id='text_output', component_property='children'),
    dash.Input(component_id='scatter_fig', component_property='hoverData')
)
def get_key_stats(hoverData):

    if not hoverData:
        return 'Hover over a country to see key stats'

    # Extract the custom data from the hoverData
    country = hoverData['points'][0]['customdata'][0]
    country_df = ecom_sales[ecom_sales['Country']==country]

    top_major_cat = country_df.groupby('Major Category')\
        .agg('size').reset_index(name='Sales Volume')\
        .sort_values(by='Sales Volume', ascending=False)\
        .reset_index(drop=True).loc[0, 'Major Category']
    top_sales_month = country_df.groupby('Year-Month')['OrderValue']\
        .agg('sum').reset_index(name='Total Sales ($)')\
        .sort_values(by='Total Sales ($)', ascending=False)\
        .reset_index(drop=True).loc[0, 'Year-Month']

    # Use the aggregated variables
    stats_list = [
        f'Key stats for : {country}',
        dash.html.Br(),
        f'The most popular Major Category by sales volume was: {top_major_cat}',
        dash.html.Br(),
        f'The highest sales value month was: {top_sales_month}'
    ]
    return stats_list

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo.png'

ecom_country = ecom_sales.groupby('Country')['OrderValue']\
    .agg(['sum', 'count']).reset_index()\
    .rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)'})
ecom_scatter = px.scatter(data_frame=ecom_country,
    x='Total Sales ($)', y='Sales Volume', color='Country',
    width=350, height=550, custom_data=['Country'])
ecom_scatter.update_layout({
    'legend':dict(orientation='h', y=-0.7, x=1, yanchor='bottom', xanchor='right')
})

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Img(src=logo_link, style={'margin':'30px 0px 0px 0px'}),
    dash.html.H1('Sales breakdowns'),
    dash.html.Div(children=[
        dash.html.Div(children=[
            dash.html.H3('Sales Volume vs Sales Amount by Country'),
            dash.dcc.Graph(id='scatter', figure=ecom_scatter),
        ], style={'width':'350px', 'height':'650px', 'display':'inline-block',
            'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
        dash.html.Div(children=[
            # Add two plot types to the layout
            dash.dcc.Graph(id='major_cat'),
            dash.dcc.Graph(id='minor_cat'),
        ], style={'width':'700px', 'height':'650px', 'display':'inline-block'})
    ])
], style={'text-align':'center', 'display':'inline-block', 'width':'100%'})

# Create a callback to update the minor category plot
@dash.callback(
    dash.Output(component_id='minor_cat', component_property='figure'),
    dash.Input(component_id='scatter', component_property='hoverData')
)
def update_min_cat_hover(hoverData):
    hover_country = 'Australia'

    if hoverData:
        hover_country = hoverData['points'][0]['customdata'][0]

    minor_cat_df = ecom_sales[ecom_sales['Country']==hover_country]
    minor_cat_agg = minor_cat_df.groupby('Minor Category')['OrderValue']\
        .agg('sum').reset_index(name='Total Sales ($)')
    ecom_bar_minor_cat = px.bar(data_frame=minor_cat_agg,
        x='Total Sales ($)', y='Minor Category', orientation='h',
        height=450, title=f'Sales by Minor Category for: {hover_country}')
    ecom_bar_minor_cat.update_layout({
        'yaxis':{'dtick':1, 'categoryorder':'total ascending'},
        'title':{'x':0.5}
    })

    return ecom_bar_minor_cat

# Create a callback to update the major category plot
@dash.callback(
    dash.Output(component_id='major_cat', component_property='figure'),
    dash.Input(component_id='scatter', component_property='hoverData')
)
def update_major_cat_hover(hoverData):
    hover_country = 'Australia'

    # Conditionally select the country from the hover data
    if hoverData:
        hover_country = hoverData['points'][0]['customdata'][0]

    major_cat_df = ecom_sales[ecom_sales['Country']==hover_country]
    major_cat_agg = major_cat_df.groupby('Major Category')['OrderValue']\
        .agg('sum').reset_index(name='Total Sales ($)')
    ecom_bar_major_cat = px.bar(data_frame=major_cat_agg,
        x='Total Sales ($)', y='Major Category', height=300,
        title=f'Sales by Major Category for: {hover_country}',
        color='Major Category', color_discrete_map={'Clothes':'blue',
            'Kitchen':'red', 'Garden':'green', 'Household':'yellow'})
    ecom_bar_major_cat.update_layout({
        'margin':dict(l=10, r=15, t=40, b=0),
        'title':{'x':0.5}
    })

    return ecom_bar_major_cat

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo.png'

ecom_country = ecom_sales.groupby('Country')['OrderValue']\
    .agg(['sum', 'count']).reset_index()\
    .rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)'})
ecom_scatter = px.scatter(data_frame=ecom_country,
    x='Total Sales ($)', y='Sales Volume', color='Country',
    width=350, height=550, custom_data=['Country'])
ecom_scatter.update_layout({
    'legend':dict(orientation='h', y=-0.7, x=1, yanchor='bottom', xanchor='right')
})

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Img(src=logo_link, style={'margin':'30px 0px 0px 0px'}),
    dash.html.H1('Sales breakdowns'),
    dash.html.Div(children=[
        dash.html.Div(children=[
            dash.html.H3('Sales Volume vs Sales Amount by Country'),
            dash.dcc.Graph(id='scatter', figure=ecom_scatter),
        ], style={'width':'350px', 'height':'650px', 'display':'inline-block',
            'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
        dash.html.Div(children=[
            dash.dcc.Graph(id='major_cat'),
            dash.dcc.Graph(id='minor_cat'),
        ], style={'width':'700px', 'height':'650px', 'display':'inline-block'})
    ])
], style={'text-align':'center', 'display':'inline-block', 'width':'100%'})

@dash.callback(
    dash.Output(component_id='major_cat', component_property='figure'),
    dash.Input(component_id='scatter', component_property='hoverData')
)
def update_major_cat_hover(hoverData):
    hover_country = 'Australia'

    if hoverData:
        hover_country = hoverData['points'][0]['customdata'][0]

    major_cat_df = ecom_sales[ecom_sales['Country']==hover_country]
    major_cat_agg = major_cat_df.groupby('Major Category')['OrderValue']\
        .agg('sum').reset_index(name='Total Sales ($)')
    ecom_bar_major_cat = px.bar(data_frame=major_cat_agg, x='Total Sales ($)',
        # Ensure the Major category will be available
        custom_data=['Major Category'], y='Major Category', height=300,
        title=f'Sales by Major Category for: {hover_country}',
        color='Major Category', color_discrete_map={'Clothes':'blue',
            'Kitchen':'red', 'Garden':'green', 'Household':'yellow'})
    ecom_bar_major_cat.update_layout({
        'margin':dict(l=10,r=15,t=40,b=0),
        'title':{'x':0.5}
    })

    return ecom_bar_major_cat

# Set up a callback for click data
@dash.callback(
    dash.Output(component_id='minor_cat', component_property='figure'),
    dash.Input(component_id='major_cat', component_property='clickData')
)
def update_major_cat_click(clickData):
    click_cat = 'All'
    major_cat_df = ecom_sales.copy()
    total_sales = major_cat_df.groupby('Country')['OrderValue']\
        .agg('sum').reset_index(name='Total Sales ($)')

    # Extract the major category clicked on for usage
    if clickData:
        click_cat = clickData['points'][0]['customdata'][0]

        # Undetake a filter using the major category clicked on
        major_cat_df = ecom_sales[ecom_sales['Major Category']==click_cat]

    country_mj_cat_agg = major_cat_df.groupby('Country')['OrderValue']\
        .agg('sum').reset_index(name='Total Sales ($)')
    country_mj_cat_agg['Sales %'] = (country_mj_cat_agg['Total Sales ($)']
        / total_sales['Total Sales ($)'] * 100).round(1)

    ecom_bar_country_mj_cat = px.bar(data_frame=country_mj_cat_agg,
        x='Sales %', y='Country', orientation='h', height=450,
        range_x = [0, 100], text='Sales %',
        title=f'Global Sales % by Country for Major Category: {click_cat}')
    ecom_bar_country_mj_cat.update_layout({
        'yaxis':{'dtick':1, 'categoryorder':'total ascending'},
        'title':{'x':0.5}
    })

    return ecom_bar_country_mj_cat

if __name__ == '__main__':
    app.run_server(debug=True)

#### Chained callbacks

In [ ]:
import dash

app = dash.Dash(__name__)

@dash.callback(
    dash.Output(component_id='minor_cat_dd', component_property='options'),
    dash.Input(component_id='major_cat_dd', component_property='value')
)
def update_dd(major_cat_dd):
    # Filter options (list of dicts)
    return minor_options

@dash.callback(
    dash.Output(component_id='minor_cat_dd', component_property='value'),
    dash.Input(component_id='minor_cat_dd', component_property='options')
)
def update_dd(minor_cat_options):
    # Pick a default value
    return chosen_value

@dash.callback(
    dash.Output(component_id='my_title', component_property='children'),
    dash.Output(component_id='minor_cat_dd', component_property='value'),
    dash.Input(component_id='minor_cat_dd', component_property='options')
)
def some_function(input):
    # function body
    return title_value, dropdown_value

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo.png'

major_categories = list(ecom_sales['Major Category'].unique())
minor_categories = list(ecom_sales['Minor Category'].unique())
ecom_country = ecom_sales.groupby('Country')['OrderValue']\
    .agg(['sum', 'count']).reset_index()
    .rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)'})

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Img(src=logo_link, style={'margin':'30px 0px 0px 0px'}),
    dash.html.H1('Sales breakdowns'),
    dash.html.Div(children=[
        dash.html.Div(children=[
            dash.html.H2('Controls'),
            dash.html.Br(),
            dash.html.H3('Major Category Select'),
            dash.dcc.Dropdown(id='major_cat_dd',
                # Set up the Major Category options with the same label and value
                options=[{'label':category, 'value':category}
                    for category in major_categories],
                style={'width':'200px', 'margin':'0 auto'}),
            dash.html.Br(),
            dash.html.H3('Minor Category Select'),
            dash.dcc.Dropdown(id='minor_cat_dd',
                style={'width':'200px', 'margin':'0 auto'})
        ], style={'width':'350px', 'height':'350px', 'display':'inline-block',
            'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
        dash.html.Div(children=[
            dash.dcc.Graph(id='sales_line')
        ], style={'width':'700px', 'height':'650px', 'display':'inline-block'})
    ])
], style={'text-align':'center', 'display':'inline-block', 'width':'100%'})

# Create a callback from the Major Category dropdown to the Minor Category Dropdown
@dash.callback(
    dash.Output(component_id='minor_cat_dd', component_property='options'),
    dash.Input(component_id='major_cat_dd', component_property='value')
)
def update_minor_dd(major_cat_dd):

    major_minor = ecom_sales[['Major Category', 'Minor Category']].drop_duplicates()
    relevant_minor_options = major_minor[major_minor['Major Category']==major_cat_dd]\
        ['Minor Category'].values.tolist()

    # Create and return formatted relevant options with the same label and value
    formatted_relevant_minor_options = [{'label':x, 'value':x}
        for x in relevant_minor_options]

    return formatted_relevant_minor_options

# Create a callback for the Minor Category dropdown to update the line plot
@dash.callback(
    dash.Output(component_id='sales_line', component_property='figure'),
    dash.Input(component_id='minor_cat_dd', component_property='value')
)
def update_line(minor_cat):
    minor_cat_title = 'All'
    ecom_line = ecom_sales.copy()

    if minor_cat:
        minor_cat_title = minor_cat
        ecom_line = ecom_line[ecom_line['Minor Category']==minor_cat]

    ecom_line = ecom_line.groupby('Year-Month')['OrderValue']\
        .agg('sum').reset_index(name='Total Sales ($)')
    line_graph = px.line(data_frame=ecom_line, x='Year-Month',  y='Total Sales ($)',
        title=f'Total Sales by Month for Minor Category: {minor_cat_title}')

    return line_graph

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import dash
import plotly.express as px
import pandas as pd
import random

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo.png'

major_categories = list(ecom_sales['Major Category'].unique())
minor_categories = list(ecom_sales['Minor Category'].unique())
ecom_country = ecom_sales.groupby('Country')['OrderValue']\
    .agg(['sum', 'count']).reset_index()
    .rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)'})

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Img(src=logo_link, style={'margin':'30px 0px 0px 0px'}),
    dash.html.H1('Sales breakdowns'),
    dash.html.Div(children=[
        dash.html.Div(children=[
            dash.html.H2('Controls'),
            dash.html.Br(),
            dash.html.H3('Major Category Select'),
            dash.dcc.Dropdown(id='major_cat_dd',
                # Set up the Major Category options with the same label and value
                options=[{'label':category, 'value':category}
                    for category in major_categories],
                style={'width':'200px', 'margin':'0 auto'}),
            dash.html.Br(),
            dash.html.H3('Minor Category Select'),
            dash.dcc.Dropdown(id='minor_cat_dd',
                style={'width':'200px', 'margin':'0 auto'})
        ], style={'width':'350px', 'height':'350px', 'display':'inline-block',
            'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
        dash.html.Div(children=[
            dash.dcc.Graph(id='sales_line'),
            dash.html.H3(id='chosen_major_cat_title')
        ], style={'width':'700px', 'height':'650px', 'display':'inline-block'})
    ])
], style={'text-align':'center', 'display':'inline-block', 'width':'100%'})

# One callback to set minor values & HTML output
@dash.callback(
    dash.Output(component_id='minor_cat_dd', component_property='options'),
    dash.Output(component_id='chosen_major_cat_title', component_property='children'),
    dash.Input(component_id='major_cat_dd', component_property='value')
)
def update_minor_dd(major_cat_dd):

    major_minor = ecom_sales[['Major Category', 'Minor Category']].drop_duplicates()
    relevant_minor_options = major_minor[major_minor['Major Category']==major_cat_dd]\
        ['Minor Category'].values.tolist()
    minor_options = [{'label':x, 'value':x} for x in relevant_minor_options]

    if not major_cat_dd:
        major_cat_dd = 'None Selected'
    # Creating string for title
    major_cat_title = f'This is in the Major Category of: {major_cat_dd}'

    # Return the options and title
    return minor_options, major_cat_title

# Create a callback to set a default minor category value
@dash.callback(
    dash.Output(component_id='minor_cat_dd', component_property='value'),
    dash.Input(component_id='minor_cat_dd', component_property='options')
)
def select_minor_cat(options):
    chosen_val = 'None'
    if options:
        vals = [x['value'] for x in options]
        chosen_val = random.choice(vals)
    return chosen_val

@dash.callback(
    dash.Output(component_id='sales_line', component_property='figure'),
    dash.Input(component_id='minor_cat_dd', component_property='value')
)
def update_line(minor_cat):
    minor_cat_title = 'All'
    ecom_line = ecom_sales.copy()

    if minor_cat:
        minor_cat_title = minor_cat
        ecom_line = ecom_line[ecom_line['Minor Category']==minor_cat]

    ecom_line = ecom_line.groupby('Year-Month')['OrderValue']\
        .agg('sum').reset_index(name='Total Sales ($)')
    line_graph = px.line(data_frame=ecom_line, x='Year-Month',  y='Total Sales ($)',
        title=f'Total Sales by Month for Minor Category: {minor_cat_title}')

    return line_graph

if __name__ == '__main__':
    app.run_server(debug=True)

#### Data table

In [ ]:
import dash

app = dash.Dash(__name__)

d_columns = [
    {'name':'Sales Volume', 'id':'Sales Volume', 'selectable':True}
]

d_table = dash.dash_table.DataTable(
    # Other table properties
    style_cell=({'textAlign':'left'}),
    style_cell_conditional=[{'if':{'column_id':'Sales Volume'}, 'textAlign':'center'}],
    style_header={'background-color':'black', 'color':'white'},
    style_header_conditional=[{'if':{'column_id':'Sales Volume'},
        'background-color':'blue'}],
    cell_selectable=True,
    row_selectable='multi',
    column_selectable='single',
    columns=d_columns
)

@dash.callback(
    dash.Output(component_id='test_text', component_property='children'),
    dash.Input(component_id='my_dt', component_property='selected_cells')
)
def print_it(input):
    return str(input)

@dash.callback(
    dash.Output(component_id='test_text', component_property='children'),
    dash.Input(component_id='my_dt', component_property='selected_rows')
)
def print_it(input):
    return str(input)

@dash.callback(
    dash.Output(component_id='test_text', component_property='children'),
    dash.Input(component_id='my_dt', component_property='selected_columns')
)
def print_it(input):
    return str(input)

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo.png'

key_stats_tb = ecom_sales.groupby(['Country', 'Major Category', 'Minor Category'])\
    ['OrderValue'].agg(['sum', 'count', 'mean']).reset_index()\
    .rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)',
        'mean':'Average Order Value ($)'})

money_format = dash.dash_table.FormatTemplate.money(2)
money_cols = ['Total Sales ($)', 'Average Order Value ($)']
data_cols = [x for x in key_stats_tb.columns if x not in money_cols]
d_columns = [{'name':x, 'id':x} for x in data_cols]
d_columns += [{'name':x, 'id':x, 'type':'numeric', 'format': money_format}
    for x in money_cols]

d_table = dash.dash_table.DataTable(columns=d_columns,
    data=key_stats_tb.to_dict('records'), cell_selectable=False, sort_action='native',
    filter_action='native', page_action='native', page_current=0, page_size=10,
    # Align all cell contents left
    style_cell=({'textAlign':'left'}),
    # Style the background of money columns
    style_cell_conditional=[
        {'if':{'column_id':'Total Sales ($)'},
            'background-color':'rgb(252, 252, 184)', 'textAlign':'center'},
        {'if':{'column_id':'Average Order Value ($)'},
            'background-color':'rgb(252, 252, 184)', 'textAlign':'center'}],
    # Style all headers
    style_header={'background-color':'rgb(168, 255, 245)'},
    # Style money header columns
    style_header_conditional=[
        {'if':{'column_id':'Total Sales ($)'},
            'background-color':'rgb(252, 252, 3)'},
        {'if':{'column_id':'Average Order Value ($)'},
            'background-color':'rgb(252, 252, 3)'}]
)

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Img(src=logo_link, style={'margin':'30px 0px 0px 0px'}),
    dash.html.H1('Sales Aggregations'),
    dash.html.Div(children=[
        dash.html.H2('Key Aggregations'),
        d_table
    ], style={'width':'850px', 'height':'750px', 'margin':'0 auto'}),
], style={'text-align':'center', 'display':'inline-block', 'width':'100%'})

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import dash
import plotly.express as px
import pandas as pd

ecom_sales = pd.read_csv('ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/e-comlogo.png'

major_categories = list(ecom_sales['Major Category'].unique())
large_tb = ecom_sales.groupby(['Country'])['OrderValue']\
    .agg(['sum', 'count', 'mean', 'median']).reset_index()\
    .rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)',
        'mean':'Average Order Value ($)', 'median':'Median Order Value ($)'})
ecom_country = ecom_sales.groupby('Country')['OrderValue']\
    .agg('sum').reset_index(name='Total Sales ($)')
bar_fig_country = px.bar(data_frame=ecom_country,
    x='Total Sales ($)', y='Country', width=500, height=450,
    title='Total Sales by Country (Hover to filter the Minor Category bar chart!)',
    custom_data=['Country'], color='Country', color_discrete_map={
        'United Kingdom':'lightblue', 'Germany':'orange',
        'France':'darkblue', 'Australia':'green', 'Hong Kong':'red'})

money_format = dash.dash_table.FormatTemplate.money(2)
money_cols = ['Total Sales ($)', 'Average Order Value ($)', 'Median Order Value ($)']
d_columns = [{'name':x, 'id':x} for x in large_tb.columns if x not in money_cols]
d_columns += [
    {'name':'Total Sales ($)', 'id':'Total Sales ($)',
        'type':'numeric', 'format':money_format,
        # Allow columns to be selected
        'selectable':True},
    {'name':'Average Order Value ($)', 'id':'Average Order Value ($)',
        'type':'numeric', 'format':money_format,
        # Allow columns to be selected
        'selectable':True},
    {'name':'Median Order Value ($)', 'id':'Median Order Value ($)',
        'type':'numeric', 'format':money_format,
        # Allow columns to be selected
        'selectable':True}
]

d_table = dash.dash_table.DataTable(id='my_dt', columns=d_columns,
    data=large_tb.to_dict('records'), cell_selectable=False,
    sort_action='native',
    # Make single columns selectable
    column_selectable='single'
)

app = dash.Dash(__name__)

app.layout = dash.html.Div(children=[
    dash.html.Img(src=logo_link, style={'margin':'30px 0px 0px 0px'}),
    dash.html.H1('Sales breakdowns'),
    dash.html.Div(children=[
        dash.html.Div(children=[
            dash.html.H2('Controls'),
            dash.html.Br(),
            dash.html.H3('Major Category Select'),
            dash.dcc.Dropdown(id='major_cat_dd',
                options=[{'label':category, 'value':category}
                    for category in major_categories],
                style={'width':'200px', 'margin':'0 auto'}),
            dash.html.Br(),
            dash.html.H3('Minor Category Select'),
            dash.dcc.Dropdown(id='minor_cat_dd',
                style={'width':'200px', 'margin':'0 auto'})
        ], style={'width':'350px', 'height':'360px', 'display':'inline-block',
            'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
        dash.html.Div(children=[
            dash.html.H3(id='chosen_major_cat_title'),
            dash.dcc.Graph(id='sales_line')
        ], style={'width':'700px', 'height':'380px', 'display':'inline-block',
            'margin-bottom':'5px'})
    ]),
    dash.html.Div(
        d_table, style={'width':'1000px', 'height':'200px',
            'margin':'10px auto', 'padding-right':'30px'}),
    dash.html.Div(children=[
        dash.dcc.Graph(id='scatter_compare'),
        dash.html.Div(dash.dcc.Graph(id='major_cat', figure=bar_fig_country),
            style={'display':'inline-block'}),
        dash.html.Div(dash.dcc.Graph(id='minor_cat'), style={'display':'inline-block'})
    ], style={'width':'1000px', 'height':'650px', 'display':'inline-block'}),
], style={'text-align':'center', 'display':'inline-block', 'width':'100%'})

# Create a callback triggered by selecting a column
@dash.callback(
    dash.Output(component_id='scatter_compare', component_property='figure'),
    dash.Input(component_id='my_dt', component_property='selected_columns')
)
def table_country(selected_columns):
    comparison_col = 'Total Sales ($)'

    # Extract comparison col using its index
    if selected_columns:
        comparison_col = selected_columns[0]

    scatter_fig = px.scatter(data_frame=large_tb, x='Sales Volume',
        # Use comparison col in figure
        y=comparison_col, color='Country',
        title=f'Sales Volume vs {comparison_col} by country')

    return scatter_fig

@dash.callback(
    dash.Output(component_id='minor_cat_dd', component_property='options'),
    dash.Output(component_id='chosen_major_cat_title', component_property='children'),
    dash.Input(component_id='major_cat_dd', component_property='value')
)
def update_dd(major_cat_dd):
    major_minor = ecom_sales[['Major Category', 'Minor Category']].drop_duplicates()
    relevant_minor = major_minor[major_minor['Major Category']==major_cat_dd]\
        ['Minor Category'].values.tolist()
    minor_options = [dict(label=x, value=x) for x in relevant_minor]

    if not major_cat_dd:
        major_cat_dd = 'ALL'

    major_cat_title = f'This is in the Major Category of : {major_cat_dd}'

    return minor_options, major_cat_title

@dash.callback(
    dash.Output(component_id='sales_line', component_property='figure'),
    dash.Input(component_id='minor_cat_dd', component_property='value')
)
def update_line(minor_cat):
    minor_cat_title = 'All'
    ecom_line = ecom_sales.copy()
    if minor_cat:
        minor_cat_title = minor_cat
        ecom_line = ecom_line[ecom_line['Minor Category']==minor_cat]
    ecom_line = ecom_line.groupby('Year-Month')['OrderValue']\
        .agg('sum').reset_index(name='Total Sales ($)')
    line_graph = px.line(data_frame=ecom_line,
        x='Year-Month', y='Total Sales ($)', height=350,
        title=f'Total Sales by Month for Minor Category: {minor_cat_title}')

    return line_graph

@dash.callback(
    dash.Output(component_id='minor_cat', component_property='figure'),
    dash.Input(component_id='major_cat', component_property='hoverData')
)
def update_min_cat_hover(hoverData):
    hover_country = 'Australia'

    if hoverData:
        hover_country = hoverData['points'][0]['customdata'][0]

    minor_cat_df = ecom_sales[ecom_sales['Country']==hover_country]
    minor_cat_agg = minor_cat_df.groupby('Minor Category')['OrderValue']\
        .agg('sum').reset_index(name='Total Sales ($)')
    ecom_bar_minor_cat = px.bar(data_frame=minor_cat_agg,
        x='Total Sales ($)', y='Minor Category', orientation='h',
        height=450, width=480, title=f'Sales by Minor Category for: {hover_country}')
    ecom_bar_minor_cat.update_layout({
        'yaxis':{'dtick':1, 'categoryorder':'total ascending'},
        'title':{'x':0.5}
    })

    return ecom_bar_minor_cat

if __name__ == '__main__':
    app.run_server(debug=True)